In [28]:
import pandas as pd
import numpy as np
import os
import re

def load_and_clean_poems(data_path):

    ds = pd.read_csv(data_path)
    
    # Datayı inceldiğimde 'pp' ve 'br' gibi htmldaen kalma atıkların temizlenmesi
    def clean_poem(poem):
        poem = str(poem)  
        poem = re.sub(r"pp[\w]*", "", poem)
        poem = re.sub(r"<br>|<br/>|br", "\n", poem)
        poem = re.sub(r"\s+", " ", poem).strip()
        return poem

    ds['cleaned_poem'] = ds['cleaned_poem'].apply(clean_poem)
    return ds


data_path = "/Users/mertgenc/DataFile/RNN-LSTM-GRU/turkish_poem/train_dataset.csv"
ds_cleaned = load_and_clean_poems(data_path)

# Temizlenmiş ilk 5 şiiri yazdır
print(ds_cleaned['cleaned_poem'].head())
#print(ds_cleaned)


                                           cleaned_poem  \
0     derhal fark edilen melek o yüce erk daha gizle...   
1     tedbirini terkeyle takdir hüdanındır sen yoksu...   
2     karanlık basmadan ovalarıma kainatın duru ille...   
3     ince narin kanatlı uçurtmadır kalbin duru yağm...   
4     her gün bir yerden göçmek ne iyi her gün bir y...   
...                                                 ...   
3464  istanbulda bir fa ika fa ikayı ben koymadım or...   
3465           özgürlüğün geldiği gün o gün ölmek yasak   
3466  sevdiğim arzımı demekçin sana bülbül söylediği...   
3467  neler mi istiyorum uyaninca her sabah ne bahar...   
3468  ben gönlü temiz insana kurban olayim gezsin ba...   

                          poet  
0           Rainer Maria Rilke  
1                   Şeyh Galib  
2              Cahit Zarifoğlu  
3                Nurullah Genç  
4      Mevlana Celaleddin Rumi  
...                        ...  
3464  Hasan Hüseyin Korkmazgil  
3465              Cemal Sür

In [30]:
full_text = " / ".join(ds_cleaned['cleaned_poem'])
print(full_text[:1000])

derhal fark edilen melek o yüce erk daha gizlendiği yere girdiği gibi tertemiz alev alev ve dikilerek yalvardı bırakıp da her bir talebi şaşkını bir tacir olarak kalmasına izin verilsin diye eskiden neydiyse öyle okuyamazdı o böylesi sözün fazla gelirdi bir âlime bile melek ona yazılı sayfasını buyurganca gösteriyor gösteriyordu tekrar ediyordu ısrarla oku o melek önüne eğdi kafasını okuyandı artık o andan itibaren ve bilendi ve uyandı ve hüküm veren osman tuğlu / tedbirini terkeyle takdir hüdanındır sen yoksun o benlikler hep vehm ü gümanındır birden bire bul aşkı bu tuhfe bulanındır devran olalı devran erbabı safanındır keder neyler gam halkı cihanındır koyma kadehi elden söz piri muganındır seyrettim uşşaka mataf olmuş teklif ü tekellüften sükkanı maaf olmuş bir neşe gelüp meclis bi havf u hilaf olmuş gam sohbeti yad olmaz meşrebleri saf olmuş keder neyler gam halkı cihanındır koyma kadehi elden söz piri muganındır dil sen o dildara layık mı değilsin ya davayı mahabete sadık mı deği

In [32]:
# Karakterleri indeksliyerek metni modelin anlayabileceği hale getirmek.
chars  = sorted(list(set(full_text)))
char_to_idx = {char: idx for idx, char in enumerate(chars)}
idx_to_char = {idx: char for char, idx in char_to_idx.items()}

In [38]:
# 40 karekter uzunluğundaki bir metnin 41. kareternin tahminleme yapamyı öğrenemsi için X ve y şeklinde ayırıyoruz.
seq_lenght = 40 
sequences = []
next_chars = []

for i in range(len(full_text) - seq_lenght):
    seq = full_text[i:i+seq_lenght]
    next_char = full_text[i+seq_lenght]
    sequences.append([char_to_idx[c] for c in seq])
    next_chars.append(char_to_idx[next_char])
    

In [46]:
print(sequences[0])

[16, 17, 30, 20, 13, 24, 0, 18, 13, 30, 23, 0, 17, 16, 21, 24, 17, 26, 0, 25, 17, 24, 17, 23, 0, 27, 0, 37, 55, 15, 17, 0, 17, 30, 23, 0, 16, 13, 20, 13]


In [43]:
print(next_chars[7])

16


In [39]:
import numpy as np
from tensorflow.keras.utils import to_categorical

X = np.array(sequences)
y = to_categorical(next_chars, num_classes=len(chars))